<a href="https://colab.research.google.com/github/kennedynjoroge/10-steps-to-become-a-data-scientist/blob/master/Forex_Algorithmic_Trading_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Business Understanding

Assumptions
- Perfect/fair market - no unfair practices by traders

## Data Importation


Import Libraries First

In [0]:
import datetime  as dt
import numpy as np
import pandas as pd
import plotly.graph_objects as go 
#import cufflinks as cf #Dynamic Plotting for plt
import matplotlib.pyplot as plt
#cf.set_config_file(offline=True)
plt.style.use('seaborn')
%matplotlib inline

#Modelling
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn import svm
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer

In [0]:
## FXCM FOREX API
!pip  install fxcmpy -q
!pip install python-socketio -q
import fxcmpy
# api token/key
api = fxcmpy.fxcmpy(config_file='fxcm.cfg')

## Quandl API Curreny API
!pip install quandl -q #q is for silent install.
import quandl
# api token/key
quandl.ApiConfig.api_key = "Y4Z_EXQ7qh7xxJhC8J6E"

View Instruments


In [3]:
instruments = api.get_instruments()
print(instruments)

['EUR/USD', 'USD/JPY', 'GBP/USD', 'USD/CHF', 'EUR/CHF', 'AUD/USD', 'USD/CAD', 'NZD/USD', 'EUR/GBP', 'EUR/JPY', 'GBP/JPY', 'CHF/JPY', 'GBP/CHF', 'EUR/AUD', 'EUR/CAD', 'AUD/CAD', 'AUD/JPY', 'CAD/JPY', 'NZD/JPY', 'GBP/CAD', 'GBP/NZD', 'GBP/AUD', 'AUD/NZD', 'USD/SEK', 'EUR/SEK', 'EUR/NOK', 'USD/NOK', 'USD/MXN', 'AUD/CHF', 'EUR/NZD', 'USD/ZAR', 'USD/HKD', 'ZAR/JPY', 'USD/TRY', 'EUR/TRY', 'NZD/CHF', 'CAD/CHF', 'NZD/CAD', 'TRY/JPY', 'USD/CNH', 'AUS200', 'ESP35', 'FRA40', 'GER30', 'HKG33', 'JPN225', 'NAS100', 'SPX500', 'UK100', 'US30', 'Copper', 'CHN50', 'EUSTX50', 'USDOLLAR', 'US2000', 'USOil', 'UKOil', 'SOYF', 'NGAS', 'WHEATF', 'CORNF', 'Bund', 'XAU/USD', 'XAG/USD', 'EMBasket', 'JPYBasket', 'BTC/USD', 'BCH/USD', 'ETH/USD', 'LTC/USD', 'XRP/USD', 'CryptoMajor', 'EOS/USD', 'XLM/USD', 'USEquities']


### Bid and Ask Historical Data

Data for today


In [4]:
candles = api.get_candles('USD/JPY', period='H1', number=10)
candles

,bidopen,bidclose,bidhigh,bidlow,askopen,askclose,askhigh,asklow,tickqty
date,,,,,,,,,
2020-04-02 06:00:00,107.273,107.325,107.358,107.139,107.289,107.341,107.372,107.153,19683
2020-04-02 07:00:00,107.325,107.283,107.356,107.173,107.341,107.297,107.371,107.184,19790
2020-04-02 08:00:00,107.283,107.301,107.409,107.249,107.297,107.315,107.422,107.260,14970
2020-04-02 09:00:00,107.301,107.294,107.338,107.247,107.315,107.307,107.353,107.260,14350
2020-04-02 10:00:00,107.294,107.422,107.441,107.283,107.307,107.434,107.452,107.296,11981
2020-04-02 11:00:00,107.422,107.322,107.423,107.289,107.434,107.335,107.439,107.301,12241
2020-04-02 12:00:00,107.322,107.181,107.411,107.011,107.335,107.198,107.425,107.027,28859
2020-04-02 13:00:00,107.181,107.257,107.371,107.118,107.198,107.276,107.384,107.133,37155
2020-04-02 14:00:00,107.257,107.987,107.992,107.237,107.276,108.003,108.009,107.252,33534


In [5]:
data_201920 = api.get_candles('USD/JPY', period = 'H1', start = dt.datetime(2019, 3, 30), stop = dt.datetime(2020, 3, 30))
data_201819 = api.get_candles('USD/JPY', period = 'H1', start = dt.datetime(2018, 3, 30), stop = dt.datetime(2019, 3, 30))
data_201718 = api.get_candles('USD/JPY', period = 'H1', start = dt.datetime(2017, 3, 30), stop = dt.datetime(2018, 3, 30))
data_201617 = api.get_candles('USD/JPY', period = 'H1', start = dt.datetime(2016, 3, 30), stop = dt.datetime(2017, 3, 30))
data_201516 = api.get_candles('USD/JPY', period = 'H1', start = dt.datetime(2015, 3, 30), stop = dt.datetime(2016, 3, 30))
data_201415 = api.get_candles('USD/JPY', period = 'H1', start = dt.datetime(2014, 3, 30), stop = dt.datetime(2015, 3, 30))
data_201314 = api.get_candles('USD/JPY', period = 'H1', start = dt.datetime(2013, 3, 30), stop = dt.datetime(2014, 3, 30))
data_201213 = api.get_candles('USD/JPY', period = 'H1', start = dt.datetime(2012, 3, 30), stop = dt.datetime(2013, 3, 30))
data_201112 = api.get_candles('USD/JPY', period = 'H1', start = dt.datetime(2011, 3, 30), stop = dt.datetime(2012, 3, 30))
data_201011 = api.get_candles('USD/JPY', period = 'H1', start = dt.datetime(2010, 3, 30), stop = dt.datetime(2011, 3, 30))

#Concatenate data for all the 10 years
years_list = [data_201920, data_201819, data_201718, data_201617, data_201516, data_201415, data_201314, data_201213, data_201112, data_201011]
df = pd.concat(years_list)

#Print Count
print("2019-2020",len(data_201920))
print("2018-2019",len(data_201819))
print("2017-2018",len(data_201718))
print("2016-2017",len(data_201617))
print("2015-2016",len(data_201516))
print("2014-2015",len(data_201415))
print("2013-2014",len(data_201314))
print("2012-2013",len(data_201213))
print("2011-2012",len(data_201112))
print("2010-2011",len(data_201011))
print("Merged Data",len(df))

#View Sample
df.head()

2019-2020 6300
2018-2019 6315
2017-2018 6331
2016-2017 6306
2015-2016 6348
2014-2015 6307
2013-2014 6296
2012-2013 6217
2011-2012 6281
2010-2011 6247
Merged Data 62948


,bidopen,bidclose,bidhigh,bidlow,askopen,askclose,askhigh,asklow,tickqty
date,,,,,,,,,
2019-03-31 18:00:00,110.847,110.954,110.993,110.923,110.886,110.993,110.998,110.970,193
2019-03-31 19:00:00,110.954,111.007,111.018,110.940,110.993,111.012,111.030,110.974,215
2019-03-31 20:00:00,111.007,111.013,111.091,110.987,111.012,111.074,111.107,111.012,321
2019-03-31 21:00:00,111.013,110.932,111.041,110.878,111.074,110.965,111.086,110.893,1320
2019-03-31 22:00:00,110.932,110.936,110.983,110.897,110.965,110.950,110.999,110.911,7250


In [6]:
df.describe()

,bidopen,bidclose,bidhigh,bidlow,askopen,askclose,askhigh,asklow,tickqty
count,62948.000000,62948.000000,62948.000000,62948.000000,62948.000000,62948.000000,62948.000000,62948.000000,62948.000000
mean,101.705842,101.706057,101.782584,101.626552,101.721464,101.721679,101.797261,101.641694,7698.083291
std,13.967212,13.967183,13.974142,13.959546,13.967709,13.967681,13.974794,13.959669,8652.560999
min,75.671000,75.671000,75.746000,75.559000,75.705000,75.705000,75.759000,75.575000,1.000000
25%,88.487250,88.487250,88.568750,88.379750,88.501000,88.501000,88.581000,88.394000,2203.000000
50%,106.286000,106.286000,106.369500,106.203500,106.301500,106.301500,106.386000,106.220000,5217.000000
75%,111.726000,111.726000,111.793250,111.659000,111.740000,111.740000,111.808000,111.674250,10287.250000
max,125.676000,125.676000,125.850000,125.602000,125.689000,125.689000,125.864000,125.617000,171038.000000


### Interest Rates

In [7]:
# Read US Interest rates from quandl for the last 10 years
# API frequency is daily
v_start_date = '2010-03-30'
v_end_date = '2020-03-30'
df_US_Interest_Rates = quandl.get("FRED/DFF", start_date=v_start_date, end_date=v_end_date)
df_US_Interest_Rates.columns = df_US_Interest_Rates.columns.str.replace('Value', 'US_Monthly_Interest_Rate')
df_JPY_Interest_Rates = quandl.get("MOFJ/INTEREST_RATE_JAPAN_40Y", start_date=v_start_date, end_date=v_end_date)
df_JPY_Interest_Rates.columns = df_JPY_Interest_Rates.columns.str.replace('Value', 'JPY_Monthly_Interest_Rate')

#Create transaction date from index
df['Transaction_Date'] = df.index.strftime('%Y-%m-%d')
df['index_num'] = df.index
df_US_Interest_Rates['Transaction_Date']  = df_US_Interest_Rates.index.strftime('%Y-%m-%d')
df_JPY_Interest_Rates['Transaction_Date']  = df_JPY_Interest_Rates.index.strftime('%Y-%m-%d')

df_JPY_Interest_Rates[0:3]

,JPY_Monthly_Interest_Rate,Transaction_Date
Date,,
2010-03-30,2.320,2010-03-30
2010-03-31,2.308,2010-03-31
2010-04-01,2.285,2010-04-01


In [0]:
df = df.merge(df_US_Interest_Rates,on='Transaction_Date',how='left')
# df.set_index("index_num", inplace = True) 
# df = df.merge(df_JPY_Interest_Rates,on='Transaction_Date',how='left')

# print (pd.merge(df2, df1, how='outer', on='month'))

In [9]:
df.set_index("index_num", inplace = True) 
df[0:3]

,bidopen,bidclose,bidhigh,bidlow,askopen,askclose,askhigh,asklow,tickqty,Transaction_Date,US_Monthly_Interest_Rate
index_num,,,,,,,,,,,
2019-03-31 18:00:00,110.847,110.954,110.993,110.923,110.886,110.993,110.998,110.970,193,2019-03-31,2.43
2019-03-31 19:00:00,110.954,111.007,111.018,110.940,110.993,111.012,111.030,110.974,215,2019-03-31,2.43
2019-03-31 20:00:00,111.007,111.013,111.091,110.987,111.012,111.074,111.107,111.012,321,2019-03-31,2.43


### Unemployment rates

In [10]:
#  % of Total Labor Force. API Frequency is annual. 
US_Unemployment_percent = quandl.get("ODA/JPN_LUR", start_date=v_start_date, end_date=v_end_date)
JPY_Unemployment_percent = quandl.get("ODA/USA_LUR", start_date=v_start_date, end_date=v_end_date)
# quandl.get("ODA/USA_LUR", authtoken="Y4Z_EXQ7qh7xxJhC8J6E")
US_Unemployment_percent

,Value
Date,
2010-12-31,5.058
2011-12-31,4.583
2012-12-31,4.325
2013-12-31,4.008
2014-12-31,3.583
2015-12-31,3.375
2016-12-31,3.108
2017-12-31,2.817
2018-12-31,2.442


## Feature Engineering

In [0]:
##Spread
#A spread is the price difference between where a trader may purchase or sell an underlying asset. 
df['spread']= df['askclose'] - df['bidclose']

## Bid Close Return
# References: https://quant.stackexchange.com/questions/21092/calculating-log-returns-across-multiple-securities-and-time , https://stackoverflow.com/questions/31742545/python-calculating-log-returns-of-a-time-series
# NB: In Quantitative Finance, doing your math in log-returns considered good manners
df['bid_close_return'] = np.log(df['bidclose']/df['bidclose'].shift(1))

## Bid Close Up or Down
#Was the bid close price up or down
df['bid_close_up_down'] = np.where(df['bid_close_return'] >0 ,1,0) # Bid Close went up or down?

##Simple Moving Average
df['bid_close_rolling_mean_1'] = df['bidclose'].rolling(window=20).mean()
df['bid_close_rolling_mean_2'] = df['bidclose'].rolling(window=50).mean()

## Bid High Return
# References: https://quant.stackexchange.com/questions/21092/calculating-log-returns-across-multiple-securities-and-time , https://stackoverflow.com/questions/31742545/python-calculating-log-returns-of-a-time-series
# NB: In Quantitative Finance, doing your math in log-returns considered good manners
df['bid_high_return'] = np.log(df['bidhigh']/df['bidhigh'].shift(1))

## Bid Close Up or Down
#Was the bid close price up or down
df['bid_high_up_down'] = np.where(df['bid_high_return'] >0 ,1,0) # Bid Close went up or down?

##Simple Moving Average
df['bid_high_rolling_mean_1'] = df['bidhigh'].rolling(window=20).mean()
df['bid_high_rolling_mean_2'] = df['bidhigh'].rolling(window=50).mean()

df['bid_high_next'] = df['bidhigh'].shift(1)

In [12]:
df = df.drop(['Transaction_Date'], axis=1)
df[0:10]

,bidopen,bidclose,bidhigh,bidlow,askopen,askclose,askhigh,asklow,tickqty,US_Monthly_Interest_Rate,spread,bid_close_return,bid_close_up_down,bid_close_rolling_mean_1,bid_close_rolling_mean_2,bid_high_return,bid_high_up_down,bid_high_rolling_mean_1,bid_high_rolling_mean_2,bid_high_next
index_num,,,,,,,,,,,,,,,,,,,,
2019-03-31 18:00:00,110.847,110.954,110.993,110.923,110.886,110.993,110.998,110.970,193,2.43,0.039,NaN,0,NaN,NaN,NaN,0,NaN,NaN,NaN
2019-03-31 19:00:00,110.954,111.007,111.018,110.940,110.993,111.012,111.030,110.974,215,2.43,0.005,0.000478,1,NaN,NaN,0.000225,1,NaN,NaN,110.993
2019-03-31 20:00:00,111.007,111.013,111.091,110.987,111.012,111.074,111.107,111.012,321,2.43,0.061,0.000054,1,NaN,NaN,0.000657,1,NaN,NaN,111.018
2019-03-31 21:00:00,111.013,110.932,111.041,110.878,111.074,110.965,111.086,110.893,1320,2.43,0.033,-0.000730,0,NaN,NaN,-0.000450,0,NaN,NaN,111.091
2019-03-31 22:00:00,110.932,110.936,110.983,110.897,110.965,110.950,110.999,110.911,7250,2.43,0.014,0.000036,1,NaN,NaN,-0.000522,0,NaN,NaN,111.041
2019-03-31 23:00:00,110.936,110.982,110.995,110.869,110.950,110.995,111.008,110.878,10756,2.43,0.013,0.000415,1,NaN,NaN,0.000108,1,NaN,NaN,110.983
2019-04-01 00:00:00,110.982,111.075,111.080,110.955,110.995,111.086,111.092,110.968,14499,2.41,0.011,0.000838,1,NaN,NaN,0.000766,1,NaN,NaN,110.995
2019-04-01 01:00:00,111.075,111.145,111.175,111.040,111.086,111.158,111.187,111.053,11333,2.41,0.013,0.000630,1,NaN,NaN,0.000855,1,NaN,NaN,111.080
2019-04-01 02:00:00,111.145,111.118,111.179,111.097,111.158,111.132,111.192,111.110,9539,2.41,0.014,-0.000243,0,NaN,NaN,0.000036,1,NaN,NaN,111.175


In [15]:
from sklearn.impute import KNNImputer
nan = np.nan
imputer = KNNImputer(n_neighbors=2, weights="uniform")
imputer.fit_transform(df)

array([[110.847   , 110.954   , 110.993   , ..., 111.045575, 110.71579 ,
        111.1075  ],
       [110.954   , 111.007   , 111.018   , ..., 111.4822  , 111.50839 ,
        110.993   ],
       [111.007   , 111.013   , 111.091   , ..., 110.85925 , 110.63102 ,
        111.018   ],
       ...,
       [ 82.464   ,  82.417   ,  82.464   , ...,  82.1658  ,  81.9     ,
         82.475   ],
       [ 82.417   ,  82.498   ,  82.525   , ...,  82.2045  ,  81.92116 ,
         82.464   ],
       [ 82.498   ,  82.454   ,  82.506   , ...,  82.24355 ,  81.94174 ,
         82.525   ]])

In [14]:
### Impute feature engineered missing values with mean
# Create an imputer object that looks for 'Nan' values, then replaces them with the mean value of the feature by columns (axis=0)
mean_imputer = SimpleImputer(missing_values='NaN', strategy='mean')
# Train the imputor on the df dataset
mean_imputer = mean_imputer.fit(df)
# Apply the imputer to the df dataset
df = mean_imputer.transform(df.values)


ValueError: ignored

## Exploratory Data Analysis

### Data Cleaning

a) Missing records

In [0]:
pass

b) Duplicate records

In [0]:
pass

c) Outliers

In [0]:
df.index

###  Univariate Analysis

In [0]:
df['bidclose'].plot()
plt.title("Bid Close ")
plt.show()

df['Bid_Close_rolling_mean_2'].plot()
plt.title("Bid Close Rolling Mean ")
plt.show()

In [0]:
#Create Plot
fig = go.Figure(data=[go.Candlestick(x=df.index,open=df['bidopen'],high=df['bidhigh'],low=df['bidlow'],close=df['bidclose'])])
#Specify title and y axis
fig.update_layout( title='Forex Pricing Patterns for last 10 years',yaxis_title='Price',xaxis_title='Year')
#pio.write_html(fig, file='../forex.html')
fig.show(renderer = "colab",auto_open=True)
# fig.write_html('tmp.html', auto_open=True)

Price was lowest in 2012 and highest in 2016


## Bivariate Analysis

## Multivariate Analysis

## Pre-Modelling Steps

a) Normality Test

b) Scaling and train test split

In [0]:
columns = ['bidopen', 'bidclose', 'bidhigh', 'bidlow', 'askopen', 'askclose','askhigh', 'asklow', 'tickqty', 
       'US_Monthly_Interest_Rate', 'spread']
labels = df['bid_close_up_down'].values #bid_high_next 'bid_close_return',  'bid_high_up_down'
features = df[list(columns)].values

min_max = MinMaxScaler()
newfeatures = min_max.fit_transform(features)
X_train, X_test, y_train, y_test = train_test_split(newfeatures, labels, test_size=0.1)

In [16]:
df.info()
#Drop null or  impute.

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 62948 entries, 2019-03-31 18:00:00 to 2011-03-30 00:00:00
Data columns (total 20 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   bidopen                   62948 non-null  float64
 1   bidclose                  62948 non-null  float64
 2   bidhigh                   62948 non-null  float64
 3   bidlow                    62948 non-null  float64
 4   askopen                   62948 non-null  float64
 5   askclose                  62948 non-null  float64
 6   askhigh                   62948 non-null  float64
 7   asklow                    62948 non-null  float64
 8   tickqty                   62948 non-null  int64  
 9   US_Monthly_Interest_Rate  62948 non-null  float64
 10  spread                    62948 non-null  float64
 11  bid_close_return          62947 non-null  float64
 12  bid_close_up_down         62948 non-null  int64  
 13  bid_close_rolling_mean_1  

## Modelling

Support Vector Machine


In [22]:
clf = svm.SVC(kernel='linear')
clf = clf.fit(X_train, y_train)

accuracy = clf.score(X_train, y_train)
print (' training data accuracy ', accuracy*100)

accuracy = clf.score(X_test, y_test)
print (' testing data accuracy ', accuracy*100)

ypredict = clf.predict(X_train)
print ('\n Training classification report\n', classification_report(y_train, ypredict))

ypredict = clf.predict(X_test)
print ('\n Testing classification report\n', classification_report(y_test, ypredict))

 traning data accuracy  53.89652798616137
 testing data accuracy  52.851469420174745

 Training classification report
               precision    recall  f1-score   support

           0       0.83      0.08      0.15     27957
           1       0.52      0.98      0.68     28696

    accuracy                           0.54     56653
   macro avg       0.68      0.53      0.42     56653
weighted avg       0.68      0.54      0.42     56653


 Testing classification report
               precision    recall  f1-score   support

           0       0.80      0.07      0.13      3139
           1       0.52      0.98      0.68      3156

    accuracy                           0.53      6295
   macro avg       0.66      0.53      0.40      6295
weighted avg       0.66      0.53      0.41      6295



Arima

In [0]:
from statsmodels.tsa.arima_model import ARIMA
# fit model
model = ARIMA(series, order=(5,1,0))
model_fit = model.fit(disp=0)
print(model_fit.summary())

Gradient Boosting

In [0]:
from sklearn.ensemble import GradientBoostingClassifier
clf = GradientBoostingClassifier(random_state=0, learning_rate=0.01, n_estimators=10000)

clf.fit(X_train, y_train)

accuracy = clf.score(X_train, y_train)
print('Testing Accuracy: %f' % accuracy)

In [23]:
y_test

array([1, 0, 1, ..., 1, 1, 1])

In [0]:
#Predict test data

pred = clf.predict(y_test)
# Calculate equity..
contracts  = 10000.0
commission = 0.0

df_trade = pd.DataFrame(train_x[train_len:,-1], columns=['return'])
df_trade['label']  = train_y[train_len:]
df_trade['pred']   = pred
df_trade['won']    = df_trade['label'] == df_trade['pred']
df_trade['return'] = df_trade['return'].shift(-1) * return_range
df_trade.drop(df_trade.index[len(df_trade)-1], inplace=True)

def calc_profit(row):
    if row['won']:
        return abs(row['return'])*contracts - commission
    else:
        return -abs(row['return'])*contracts - commission

df_trade['pnl'] = df_trade.apply(lambda row: calc_profit(row), axis=1)
df_trade['equity'] = df_trade['pnl'].cumsum()

display(df_trade.tail())
df_trade.plot(y='equity', figsize=(10,4), title='Backtest with $10000 initial capital')
plt.xlabel('Trades')
plt.ylabel('Equity (USD)')
for r in df_trade.iterrows():
    if r[1]['won']:
        plt.axvline(x=r[0], linewidth=0.5, alpha=0.8, color='g')
    else:
        plt.axvline(x=r[0], linewidth=0.5, alpha=0.8, color='r')
